In [1]:
import torch
from detanet_model import nn_vib_analysis,Lorenz_broadening,DetaNet,get_raman_intensity,uv_model,nmr_calculator,nmr_sca,charge_model, polar_model

In [2]:
device=torch.device('cpu')
dtype=torch.float32

import detanet_model as dm

In [47]:
'''
The spectrum calculation modules of DetaNet consists of a tensor of the shape [Num_atom,3] 
indicating the position of the atom and a LongTensor of [Num_atom] indicating the element type.
Here, we take the example of Phenol
'''
pos=torch.tensor([[ 9.7233e-02,  1.3689e+00,  1.3115e-01],
        [ 6.7777e-02,  3.2910e-03,  3.8910e-02],
        [ 7.6066e-02, -6.6728e-01, -1.1832e+00],
        [ 4.4743e-02, -2.0582e+00, -1.2085e+00],
        [ 5.2200e-03, -2.7850e+00, -2.4071e-02],
        [-2.7410e-03, -2.1055e+00,  1.1924e+00],
        [ 2.8234e-02, -7.1819e-01,  1.2309e+00],
        [ 1.2295e-01,  1.7542e+00, -7.5192e-01],
        [ 1.0686e-01, -1.0483e-01, -2.1109e+00],
        [ 5.1425e-02, -2.5716e+00, -2.1622e+00],
        [-1.9064e-02, -3.8668e+00, -4.6867e-02],
        [-3.3378e-02, -2.6606e+00,  2.1220e+00],
        [ 2.2412e-02, -1.7973e-01,  2.1697e+00]],device=device,dtype=dtype)

z=torch.LongTensor([8, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1])

pos.shape,z.shape

(torch.Size([13, 3]), torch.Size([13]))

In [54]:
polar_model_instance = polar_model(device=device)

polar_model_instance

/media/maria/work_space/capsule-3259363/code/detanet_model/model_loader.py:104: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



DetaNet(
  (Embedding): Embedding(
    (act): Swish()
    (elec_emb): Linear(in_features=16, out_features=128, bias=False)
    (nuclare_emb): Embedding(10, 128)
    (ls): Linear(in_features=128, out_features=128, bias=True)
  )
  (Radial): Radial_Basis(
    (radial): Bessel_Function()
  )
  (blocks): Sequential(
    (0): Interaction_Block(
      (message): Message(
        (Attention): Edge_Attention(
          (actq): Swish()
          (actk): Swish()
          (actv): Swish()
          (acta): Swish()
          (softmax): Softmax(dim=-1)
          (lq): Linear(in_features=128, out_features=128, bias=True)
          (lk): Linear(in_features=128, out_features=128, bias=True)
          (lv): Linear(in_features=128, out_features=256, bias=True)
          (la): Linear(in_features=256, out_features=256, bias=True)
          (lrbf): Linear(in_features=32, out_features=128, bias=False)
          (lkrbf): Linear(in_features=128, out_features=128, bias=False)
          (lvrbf): Linear(in_featu

In [49]:
import torch_cluster
import ase
from ase.io import read
from ase.visualize import view

In [50]:
polar = polar_model_instance(pos= pos, z= z)

polar

tensor([[39.1609,  1.1352, -0.8025],
        [ 1.1352, 87.7239, -1.2346],
        [-0.8025, -1.2346, 79.1886]], grad_fn=<MulBackward0>)

In [51]:
from ase.build import molecule

dataset = torch.load("../data/qm9s.pt")
print(dataset[0])



/tmp/ipykernel_14364/1263027243.py:3: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



KeyboardInterrupt: 

In [65]:
import torch_geometric

test_loader = torch_geometric.loader.DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
from code.util.visualize_polarizability import smiles_to_atoms, visualize_polarizability, compare_polarizabilities_eigen
from ase.io import read
from ase.visualize import view

# Example usage
sample = dataset[0]

smiles = sample.smile
print(smiles)

atoms = smiles_to_atoms(smiles)

# Visualize the molecule
view(atoms, viewer='x3d')

C


In [67]:

print(dataset[1])

# Example polarizability tensor
polarizability = sample.polar

print("smiles code", sample.smile)
print("polarizability", polarizability)
# Visualize
visualize_polarizability(atoms, polarizability)


Data(edge_index=[2, 12], pos=[4, 3], number=2, smile='N', z=[4], quadrupole=[1, 3, 3], octapole=[1, 3, 3, 3], npacharge=[4], dipole=[1, 3], polar=[1, 3, 3], hyperpolar=[1, 3, 3, 3], energy=[1, 1], Hij=[12, 3, 3], Hi=[4, 3, 3], dedipole=[4, 3, 3], depolar=[4, 3, 6], tran_dipole=[1, 10, 3], tran_energy=[1, 10])
smiles code C
polarizability tensor([[[1.4457e+01, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.4457e+01, 1.2801e-05],
         [0.0000e+00, 1.2801e-05, 1.4457e+01]]])
Raw Eigenvalues (True Tensor): [14.456698 14.456802 14.4568  ]
Real Eigenvalues (True Tensor): [14.456698 14.456802 14.4568  ]


In [34]:
import torch_geometric
import copy
import torch
from torch.nn.functional import one_hot
import numpy as np

# Function to generate one-hot encodings
def generate_onehot(z, max_atomic_number):
    indices = z - 1  # Shift atomic numbers to 0-based index
    return one_hot(indices, num_classes=max_atomic_number).float()

def get_dataset(dataset):
    # Maximum atomic number in the dataset
    max_atomic_number = 9

    new_dataset = []

    for molecule in dataset:
        pos = molecule['pos']
        z = molecule['z']
        polar = molecule['polar'].squeeze()

        # Create the dataset entry
        data_entry = torch_geometric.data.Data(
            pos=pos.to(torch.float32),    # Atomic positions
            z=torch.LongTensor(z),        # Atomic numbers
            y=torch.tensor(polar, dtype=torch.float32)        # polar.to(torch.float32)  # Polarizability tensor (target)
        )

        new_dataset.append(data_entry)
    
    return new_dataset

data_nn = get_dataset(dataset)



/tmp/ipykernel_14364/62396951.py:27: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [68]:
import plotly.graph_objects as go

def compare_polarizabilities(atoms, true_tensor, predicted_tensor):
    """
    Visualize a molecule with true and predicted polarizability tensors using Plotly.
    
    Parameters:
        atoms: ASE Atoms object
        true_tensor: (3, 3) numpy array representing the true polarizability tensor (Bohr^3)
        predicted_tensor: (3, 3) numpy array representing the predicted polarizability tensor (Bohr^3)
    """
    def create_polarizability_traces(tensor, center, color, name, scale=0.05):

        tensor = tensor.reshape(3,3)

        # Eigen decomposition of the polarizability tensor
        eigenvalues, eigenvectors = np.linalg.eig(tensor)
        
        print("Raw Eigenvalues (True Tensor):", eigenvalues)
        print("Real Eigenvalues (True Tensor):", np.real(eigenvalues))
        # Ensure eigenvalues and eigenvectors are real
        eigenvalues = np.real(eigenvalues)
        eigenvectors = np.real(eigenvectors)


        # Calculate arrows for the principal axes
        arrows = [scale * eigenvalue * eigenvector for eigenvalue, eigenvector in zip(eigenvalues, eigenvectors.T)]

        # Create arrow traces
        arrow_traces = []
        for arrow in arrows:
            arrow_trace = go.Scatter3d(
                x=[center[0], center[0] + arrow[0]],
                y=[center[1], center[1] + arrow[1]],
                z=[center[2], center[2] + arrow[2]],
                mode='lines+markers',
                line=dict(color=color, width=4),
                marker=dict(size=4, color=color),
                name=f"{name} Arrow"
            )
            arrow_traces.append(arrow_trace)

        # Generate ellipsoid points
        eigenvalues = np.abs(eigenvalues) * 0.05  # Adjust scaling factor
        u = np.linspace(0, 2 * np.pi, 50)
        v = np.linspace(0, np.pi, 25)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones_like(u), np.cos(v))

        # Scale and rotate the ellipsoid
        ellipsoid = np.dot(np.column_stack([x.flatten(), y.flatten(), z.flatten()]), np.diag(eigenvalues))
        ellipsoid = np.dot(ellipsoid, eigenvectors.T)
        x_ellipsoid = ellipsoid[:, 0].reshape(x.shape)
        y_ellipsoid = ellipsoid[:, 1].reshape(y.shape)
        z_ellipsoid = ellipsoid[:, 2].reshape(z.shape)

        # Create ellipsoid trace
        ellipsoid_trace = go.Surface(
            x=x_ellipsoid + center[0],
            y=y_ellipsoid + center[1],
            z=z_ellipsoid + center[2],
            colorscale=[[0, color], [1, color]],
            opacity=0.2,
            showscale=False,
            name=f"{name} Ellipsoid"
        )
        return arrow_traces, ellipsoid_trace

    # Calculate center of the molecule
    center = atoms.get_center_of_mass()

    # Create polarizability traces for true tensor
    true_arrows, true_ellipsoid = create_polarizability_traces(true_tensor, center, 'blue', 'True')

    # Create polarizability traces for predicted tensor
    pred_arrows, pred_ellipsoid = create_polarizability_traces(predicted_tensor, center, 'red', 'Predicted')

    # Extract atomic positions and symbols
    positions = atoms.get_positions()
    symbols = atoms.get_chemical_symbols()

    # Define colors for different atom types
    atom_colors = {
        'H': 'grey',
        'C': 'black',
        'O': 'pink',
        'N': 'purple',
        'S': 'yellow',
        'Cl': 'green',
        'F': 'cyan'
    }
    colors = [atom_colors.get(symbol, 'gray') for symbol in symbols]

    # Create scatter plot for the atoms with different colors
    atom_trace = go.Scatter3d(
        x=positions[:, 0],
        y=positions[:, 1],
        z=positions[:, 2],
        mode='markers+text',
        marker=dict(size=6, color=colors),
        text=symbols,
        textposition="top center",
        name="Atoms"
    )

    # Combine all traces
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        title="True vs Predicted Polarizability Comparison"
    )

    fig = go.Figure(data=[atom_trace, true_ellipsoid, pred_ellipsoid] + true_arrows + pred_arrows, layout=layout)

    # Display the plot
    fig.show()

In [53]:
import random
from ase import Atoms

def visualize_random_predictions(model, dataset, num_samples=3):
    """
    Visualize random predictions of the model by comparing eigenvalues and eigenvectors
    of true and predicted polarizability tensors.
    
    Parameters:
        model: Trained model
        dataset: Dataset containing test samples
        num_samples: Number of random samples to visualize
    """
    model.eval()  # Set model to evaluation mode
    random_indices = random.sample(range(len(dataset)), num_samples)  # Randomly select samples
    
    with torch.no_grad():
        for idx in random_indices:
            data = dataset[idx].to('cpu')  # Get a random data point and send it to CUDA
            output = model(pos= data.pos, z = data.z)
            
            # Reshape the predicted output into a 3x3 tensor
            predicted_polarizability = output.cpu().numpy()            
            # Perform eigen decomposition for the true tensor
            true_polarizability = data.y.cpu().numpy()
            
            # Prepare ASE Atoms object
            atomic_positions = data.pos.cpu().numpy()
            atomic_numbers = data.z.cpu().numpy()
            atoms = Atoms(numbers=atomic_numbers, positions=atomic_positions)

            center = atoms.get_center_of_mass()
            # Visualize using the eigenvalue and eigenvector comparison function
            compare_polarizabilities(
                atoms,
                true_polarizability,
                predicted_polarizability
            )
visualize_random_predictions(polar_model_instance , data_nn, num_samples=5)


Raw Eigenvalues (True Tensor): [ 55.285217  89.882355 106.04343 ]
Real Eigenvalues (True Tensor): [ 55.285217  89.882355 106.04343 ]
Raw Eigenvalues (True Tensor): [ 55.318455  89.790596 106.02717 ]
Real Eigenvalues (True Tensor): [ 55.318455  89.790596 106.02717 ]


Raw Eigenvalues (True Tensor): [96.87353  66.36554  69.060844]
Real Eigenvalues (True Tensor): [96.87353  66.36554  69.060844]
Raw Eigenvalues (True Tensor): [96.66425  66.06425  69.374596]
Real Eigenvalues (True Tensor): [96.66425  66.06425  69.374596]


Raw Eigenvalues (True Tensor): [112.73098  91.86756  88.33336]
Real Eigenvalues (True Tensor): [112.73098  91.86756  88.33336]
Raw Eigenvalues (True Tensor): [112.559006  91.9062    88.37625 ]
Real Eigenvalues (True Tensor): [112.559006  91.9062    88.37625 ]


Raw Eigenvalues (True Tensor): [95.186226 90.16855  38.76392 ]
Real Eigenvalues (True Tensor): [95.186226 90.16855  38.76392 ]
Raw Eigenvalues (True Tensor): [95.37674  89.944756 39.044334]
Real Eigenvalues (True Tensor): [95.37674  89.944756 39.044334]


Raw Eigenvalues (True Tensor): [ 69.73281  92.1113  103.88759]
Real Eigenvalues (True Tensor): [ 69.73281  92.1113  103.88759]
Raw Eigenvalues (True Tensor): [ 69.56158  92.02315 104.10504]
Real Eigenvalues (True Tensor): [ 69.56158  92.02315 104.10504]


In [69]:
from torch2jax import j2t, t2j

pom = polar_model_instance.eval()

jax_pom = t2j(pom)

jax_pom


<function torch2jax.t2j_module.<locals>.f(x, state_dict={})>

In [70]:
pom

DetaNet(
  (Embedding): Embedding(
    (act): Swish()
    (elec_emb): Linear(in_features=16, out_features=128, bias=False)
    (nuclare_emb): Embedding(10, 128)
    (ls): Linear(in_features=128, out_features=128, bias=True)
  )
  (Radial): Radial_Basis(
    (radial): Bessel_Function()
  )
  (blocks): Sequential(
    (0): Interaction_Block(
      (message): Message(
        (Attention): Edge_Attention(
          (actq): Swish()
          (actk): Swish()
          (actv): Swish()
          (acta): Swish()
          (softmax): Softmax(dim=-1)
          (lq): Linear(in_features=128, out_features=128, bias=True)
          (lk): Linear(in_features=128, out_features=128, bias=True)
          (lv): Linear(in_features=128, out_features=256, bias=True)
          (la): Linear(in_features=256, out_features=256, bias=True)
          (lrbf): Linear(in_features=32, out_features=128, bias=False)
          (lkrbf): Linear(in_features=128, out_features=128, bias=False)
          (lvrbf): Linear(in_featu

In [74]:
import torch
from torch_geometric.loader import DataLoader
from torch2jax import t2j, j2t
import jax
import jax.numpy as jnp
from ase import Atoms
import random

# Convert PyTorch model to JAX function with explicit parameters
def convert_model_to_jax(torch_model):
    """Converts a PyTorch model to a JAX-compatible function."""
    jax_model = t2j(torch_model)
    state_dict = {k: v.detach().cpu().numpy() for k, v in torch_model.state_dict().items()}
    return jax_model, state_dict

# JAX-compatible visualization function
def jax_visualize_random_predictions(model_func, params, dataloader, num_samples=3):
    """
    Visualize random predictions of the model by comparing eigenvalues and eigenvectors
    of true and predicted polarizability tensors.
    
    Parameters:
        model_func: JAX-compatible model function
        params: JAX model parameters
        dataset: Dataset containing test samples
        num_samples: Number of random samples to visualize
    """    
    for data in dataloader:

        batch = t2j(data)

        # Call the JAX model
        predicted_polarizability = model_func(batch, state_dict=params)

        # Extract true polarizability
        true_polarizability = data.y.cpu().numpy()

        # Prepare ASE Atoms object
        atomic_positions = data.pos.cpu().numpy()
        atomic_numbers = data.z.cpu().numpy()
        atoms = Atoms(numbers=atomic_numbers, positions=atomic_positions)

        # Visualize polarizability (replace with your visualization function)
        center = atoms.get_center_of_mass()
        compare_polarizabilities(
            atoms,
            true_polarizability,
            predicted_polarizability
        )

# Example usage
device = "cpu"
polar_model_instance = polar_model(device)
polar_model_instance.eval()

# Convert the model to JAX
jax_model_func, jax_params = convert_model_to_jax(polar_model_instance)

# Load your dataset
# Assuming `data_nn` is a PyTorch Geometric dataset
jax_visualize_random_predictions(jax_model_func, jax_params, test_loader, num_samples=5)


/media/maria/work_space/capsule-3259363/code/detanet_model/model_loader.py:104: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



AttributeError: 'tuple' object has no attribute 'split'